#### Installing Packages

<i>%pip install tensorflow <br>
%pip install keras<br>
%pip install matplotlib<br>
%pip install pillow<br>
%pip install tqdm<br>
%pip install -U scikit-learn</i>

In [1]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from PIL import Image, ImageFile
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
import os
from tqdm import tqdm

In [2]:
IMAGE_SIZE = [300, 400]
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

for layer in vgg.layers:
    layer.trainable = False

58900480/58889256 [==============================] - 1s 0us/step


In [3]:
x = Flatten()(vgg.output)
preds = Dense(2, activation='softmax')(x)

model = Model(inputs=vgg.input, outputs=preds)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 300, 400, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 300, 400, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 300, 400, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 150, 200, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 150, 200, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 150, 200, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 75, 100, 128)      0     

In [4]:
model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer="adam",
    metrics=['accuracy'],
)

In [5]:
ImageFile.LOAD_TRUNCATED_IMAGES = True
TRAIN_DATA_PATH = "../../my-datasets/FloodNet Challenge @ EARTHVISION 2021 - Track 1/Train/Classification"
CATEGORIES = ["Flooded", "Non-Flooded"]
train_data_list = []

for category in CATEGORIES:
    path = os.path.join(TRAIN_DATA_PATH, category)
    class_num = CATEGORIES.index(category)
    for img_path in tqdm(os.listdir(path)):
        try:
            img = Image.open(os.path.join(path, img_path))
            img = np.array(img.resize((400, 300)))
            train_data_list.append([img, class_num])
        except Exception as e:
            print(e)
print(f"Total number of samples is {len(train_data_list)}")

100%|██████████| 347/347 [01:10<00:00,  4.93it/s]

Total number of samples is 480


In [6]:
import random
random.shuffle(train_data_list)

X = []
y = []

for image, label in train_data_list:
    X.append(image)
    y.append(label)

X = np.array(X).reshape(-1, 300, 400, 3)
y = np.array(y)

#np.savez_compressed("data/X_images", X, allow_pickle=True)
#np.savez_compressed("data/y_labels", y, allow_pickle=True)

In [7]:
#X = np.load("X_images.npy")
#y = np.load("y_labels.npy")

model.fit(
    x=X,
    y=y,
    batch_size=16,
    epochs=3,
    verbose=1,
) 

2022-01-17 11:24:05.113607: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 172800000 exceeds 10% of free system memory.
2022-01-17 11:24:05.302974: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 172800000 exceeds 10% of free system memory.


Epoch 1/3
30/30 [==============================] - 12s 141ms/step - loss: 4.5586 - accuracy: 0.9021
Epoch 2/3
30/30 [==============================] - 4s 142ms/step - loss: 0.1760 - accuracy: 0.9833
Epoch 3/3
30/30 [==============================] - 4s 142ms/step - loss: 0.3731 - accuracy: 0.9875


In [8]:
#model.evaluate(x=X, y=y, batch_size=16)
preds = model.predict(x=X)
preds_ = np.argmax(preds, axis=1)

total = 0
correct = 0

for pred, truth in zip(preds_, y):
    if(pred == truth):
        correct += 1
    total += 1
    
print(f"Accuracy: {(correct/total)*100}%")

2022-01-17 11:24:40.957511: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 172800000 exceeds 10% of free system memory.
2022-01-17 11:24:41.138035: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 172800000 exceeds 10% of free system memory.


Accuracy: 100.0%
